In [5]:
#トラカンデータの距離案分()
import numpy as np
import psycopg2
import datetime
import pandas as pd

In [55]:
#計測地点のosm_idを指定
start_osm = 211152194
mid_osm = [228330076]
end_osm =  228330075
start_mesh = 50325623313
end_mesh = 50325613133

In [56]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [57]:
getdbname = "kddi_00_new_param"
dbname = "kddi_trkn00_monthly_anbun"
trknName = "trkn_00_grp"
sttime = "2022-04-01 00:00:00"

In [58]:
for i in range(30):
    dte = datetime.datetime.strptime(sttime, '%Y-%m-%d %H:%M:%S')
    if i != 0:
        dte = dte + datetime.timedelta(days=i)
    for j in range(24):
        timestr = dte.strftime('%H:%M:%S')
        #特定日付のKDDIメッシュ情報を全取得
        cur =connection.cursor()
        sql = "SELECT mesh, tbran,pointdata.latitude,pointdata.longitude FROM " + getdbname + " "
        sql += "INNER JOIN pointdata ON mesh = cast(pointdata.grid_code AS BIGINT)  "
        sql += "WHERE times = '"+str(dte)+"' AND pointdata.grid_code != '0'  ORDER BY mesh "
        cur.execute(sql)
        kddiDf = pd.DataFrame.from_records(cur.fetchall(), columns=["mesh","tbran","latitude","longitude"])
        cur.close()

        one_target_list = []
        one_target_list.append(start_mesh)
        one_target_list.append(end_mesh)
        two_target_list = []
        two_target_list.append(start_osm)
        if len(mid_osm) != 0:
            for target in mid_osm:
                two_target_list.append(target)
        if start_osm != end_osm:
            two_target_list.append(end_osm)

        cur =connection.cursor()
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(start_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            stlat = row[0]
            stlng = row[1]
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(end_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            edlat = row[0]
            edlng = row[1]
        cur.close()

        if stlat > edlat:
            wklat = stlat
            stlat = edlat
            edlat = wklat
        if stlng > edlng:
            wklng = stlng
            stlng = edlng
            edlng = wklng

        print(str(stlat) + ":" + str(edlat) + ":" + str(stlng) + ":" + str(stlng))

        keylist = []
        cnt = 0
        for item in two_target_list:
            sql = "SELECT a.grid_code,a.latitude,a.longitude"
            sql += " FROM pointdata a, roadmap r"
            sql += " WHERE osm_id = "+ str(item) +" and ST_Distance(a.geom, r.geom) < 0.001"
            sql += " order by ST_Distance(a.geom, r.geom);"
            cur =connection.cursor()
            cur.execute(sql)
            childlist = []
            for row in cur:
                if row[1] >= stlat and row[1] <= edlat and row[2] >= stlng and row[2] <= edlng:
                    childlist.append([row[0], row[1], row[2]])
                #print(row[0])
            worklist = [item, childlist]
            keylist.append(worklist)
            cnt = cnt + 1
            cur.close()
        #特定日付のトラカンデータを全取得 > 緯度経度
        cur =connection.cursor()
        #sql = "SELECT trakanmap.gridcode , 断面交通量,pointdata.latitude,pointdata.longitude FROM trakan2204 "
        #sql += "INNER JOIN trakanmap ON trakan2204.計測地点番号 = trakanmap.地点番号 "
        #sql += "INNER JOIN pointdata ON trakanmap.gridcode = pointdata.grid_code "
        #sql += "WHERE 時刻 = '"+str(dte)+"' AND (trakanmap.gridcode = '"+str(start_mesh)+"' or trakanmap.gridcode = '"+str(end_mesh)+"') "
        #sql += "ORDER BY trakanmap.gridcode"
        sql = "SELECT grid_code, tbran, latitude, longitude FROM " + trknName  + " "
        sql += "INNER JOIN pointdata ON mesh = cast(grid_code as bigint) "
        sql += "WHERE times = '"+timestr+"' AND (mesh = '"+str(start_mesh)+"' or mesh = '"+str(end_mesh)+"') "
        sql += "ORDER BY mesh"
        #print(sql)
        cur.execute(sql)
        trknDf = pd.DataFrame(cur.fetchall(), columns=["gridcode","danmen","latitude","longitude"])
        cur.close()
        trknDf
        #keylist

        from geopy.distance import geodesic

        #データが存在する道路IDごとに交通量を平均化して取得する
        lstItems = []
        cntDt = 0
        wks = ""
        wknp=""
        find = 0

        trknStParam = trknDf.query('gridcode == "'+str(start_mesh) + '"')
        trknEdParam = trknDf.query('gridcode == "'+str(end_mesh) + '"')
        trknStParam

        if len(trknStParam) == 0:
            print(str(start_mesh) + " mesh empty!")
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        elif len(trknStParam) == 1:
            trknSt = trknStParam['danmen'].iloc[-1]
        else:
            trknSt = trknStParam['danmen'].iloc[0] + trknStParam['danmen'].iloc[1]

        if len(trknEdParam) == 0:
            print(str(end_mesh) + " mesh empty!")
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        elif len(trknEdParam) == 1:
            trknEd = trknEdParam['danmen'].iloc[-1]
        else:
            trknEd = trknEdParam['danmen'].iloc[0] + trknEdParam['danmen'].iloc[1]

        st_station = [trknStParam['latitude'].iloc[-1], trknStParam['longitude'].iloc[-1]]
        ed_station = [trknEdParam['latitude'].iloc[-1], trknEdParam['longitude'].iloc[-1]]

        dis_km = geodesic(st_station, ed_station)
        if dis_km == 0:
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        cur =connection.cursor()
        for row in keylist:
            for meshid in row[1]:
                #路線のメッシュ数分ループ
                print(str(meshid[0]))
                kddiParam = kddiDf.query('mesh == '+str(meshid[0]))
                if len(kddiParam) == 0:
                    print(str(meshid[0]) + " is kddi none!")
                    continue
                #路線のメッシュがkddiに存在したなら
                
                kddi_station = [kddiParam['latitude'].iloc[-1], kddiParam['longitude'].iloc[-1]]
                kddi_km = geodesic(st_station, kddi_station)
                
                if trknSt < trknEd:
                    ret = ((trknEd - trknSt) * (kddi_km / dis_km)) + trknSt
                else:
                    ret = trknSt - ((trknEd - trknSt) * (kddi_km / dis_km))
                print("存在メッシュ：" + str(meshid[0]) + "値" + str(ret) + "距離" + str(kddi_km))
                sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + ") ON CONFLICT DO NOTHING;"
                cur.execute(sql)
                connection.commit()

        sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(start_mesh) + ","  + str(trknSt) + ") ON CONFLICT DO NOTHING;"
        cur.execute(sql)
        connection.commit()
        cur.close()
        dte = dte + datetime.timedelta(minutes=15) 
        #print(lstItems)


33.7619791665:33.7723958335:132.788281250:132.788281250
50325623133
50325623133 is kddi none!
50325623311
存在メッシュ：50325623311値6.4510722666363245距離0.11553990396805806 km
50325623313
存在メッシュ：50325623313値6.315789473684211距離0.0 km
50325623131
存在メッシュ：50325623131値6.721637654053576距離0.34661954238397397 km
50325623113
50325623113 is kddi none!
50325623111
存在メッシュ：50325623111値6.992203014924333距離0.5776991581275353 km
50325623131
存在メッシュ：50325623131値6.721637654053576距離0.34661954238397397 km
50325613333
存在メッシュ：50325613333値7.127485628584548距離0.6932389089691811 km
50325623133
50325623133 is kddi none!
50325613333
存在メッシュ：50325613333値7.127485628584548距離0.6932389089691811 km
50325613331
存在メッシュ：50325613331値7.262768284310787距離0.808778695737822 km
50325613313
存在メッシュ：50325613313値7.398050852233345距離0.9243184075166689 km
50325613311
存在メッシュ：50325613311値7.533333397285908距離1.0398580997631206 km
50325613133
存在メッシュ：50325613133値7.668615984405458距離1.1553978279373909 km
50325623111
存在メッシュ：50325623111値6.992203014924333距離

33.7619791665:33.7723958335:132.788281250:132.788281250
50325623133
50325623133 is kddi none!
50325623311
50325623311 is kddi none!
50325623313
存在メッシュ：50325623313値3.709090909090909距離0.0 km
50325623131
50325623131 is kddi none!
50325623113
50325623113 is kddi none!
50325623111
存在メッシュ：50325623111値3.7836363951422762距離0.5776991581275353 km
50325623131
50325623131 is kddi none!
50325613333
存在メッシュ：50325613333値3.798545481928314距離0.6932389089691811 km
50325623133
50325623133 is kddi none!
50325613333
存在メッシュ：50325613333値3.798545481928314距離0.6932389089691811 km
50325613331
50325613331 is kddi none!
50325613313
存在メッシュ：50325613313値3.82836365509575距離0.9243184075166689 km
50325613311
存在メッシュ：50325613311値3.8432727343207462距離1.0398580997631206 km
50325613133
存在メッシュ：50325613133値3.858181818181818距離1.1553978279373909 km
50325623111
存在メッシュ：50325623111値3.7836363951422762距離0.5776991581275353 km
33.7619791665:33.7723958335:132.788281250:132.788281250
50325623133
50325623133 is kddi none!
50325623311
503256233

33.7619791665:33.7723958335:132.788281250:132.788281250
50325623133
50325623133 is kddi none!
50325623311
存在メッシュ：50325623311値4.770000038454793距離0.11553990396805806 km
50325623313
50325623313 is kddi none!
50325623131
存在メッシュ：50325623131値4.843333394900304距離0.34661954238397397 km
50325623113
存在メッシュ：50325623113値4.88000008142497距離0.46215938775195675 km
50325623111
存在メッシュ：50325623111値4.916666744150721距離0.5776991581275353 km
50325623131
存在メッシュ：50325623131値4.843333394900304距離0.34661954238397397 km
50325613333
存在メッシュ：50325613333値4.953333400677359距離0.6932389089691811 km
50325623133
50325623133 is kddi none!
50325613333
存在メッシュ：50325613333値4.953333400677359距離0.6932389089691811 km
50325613331
存在メッシュ：50325613331値4.990000068605464距離0.808778695737822 km
50325613313
存在メッシュ：50325613313値5.026666712735483距離0.9243184075166689 km
50325613311
存在メッシュ：50325613311値5.063333350666877距離1.0398580997631206 km
50325613133
存在メッシュ：50325613133値5.1距離1.1553978279373909 km
50325623111
存在メッシュ：50325623111値4.916666744150721距離

33.7619791665:33.7723958335:132.788281250:132.788281250
50325623133
50325623133 is kddi none!
50325623311
50325623311 is kddi none!
50325623313
50325623313 is kddi none!
50325623131
存在メッシュ：50325623131値4.875000013992493距離0.34661954238397397 km
50325623113
50325623113 is kddi none!
50325623111
存在メッシュ：50325623111値4.891666684276679距離0.5776991581275353 km
50325623131
存在メッシュ：50325623131値4.875000013992493距離0.34661954238397397 km
50325613333
50325613333 is kddi none!
50325623133
50325623133 is kddi none!
50325613333
50325613333 is kddi none!
50325613331
50325613331 is kddi none!
50325613313
50325613313 is kddi none!
50325613311
存在メッシュ：50325613311値4.9250000039394415距離1.0398580997631206 km
50325613133
50325613133 is kddi none!
50325623111
存在メッシュ：50325623111値4.891666684276679距離0.5776991581275353 km
33.7619791665:33.7723958335:132.788281250:132.788281250
50325623133
50325623133 is kddi none!
50325623311
50325623311 is kddi none!
50325623313
50325623313 is kddi none!
50325623131
存在メッシュ：50325623131値

33.7619791665:33.7723958335:132.788281250:132.788281250
50325623133
50325623133 is kddi none!
50325623311
50325623311 is kddi none!
50325623313
50325623313 is kddi none!
50325623131
50325623131 is kddi none!
50325623113
50325623113 is kddi none!
50325623111
50325623111 is kddi none!
50325623131
50325623131 is kddi none!
50325613333
50325613333 is kddi none!
50325623133
50325623133 is kddi none!
50325613333
50325613333 is kddi none!
50325613331
50325613331 is kddi none!
50325613313
50325613313 is kddi none!
50325613311
存在メッシュ：50325613311値4.275272748416786距離1.0398580997631206 km
50325613133
50325613133 is kddi none!
50325623111
50325623111 is kddi none!
33.7619791665:33.7723958335:132.788281250:132.788281250
50325623133
50325623133 is kddi none!
50325623311
50325623311 is kddi none!
50325623313
50325623313 is kddi none!
50325623131
50325623131 is kddi none!
50325623113
50325623113 is kddi none!
50325623111
50325623111 is kddi none!
50325623131
50325623131 is kddi none!
50325613333
503256

...

33.7619791665:33.7723958335:132.788281250:132.788281250
50325623133
50325623133 is kddi none!
50325623311
存在メッシュ：50325623311値8.183576870412207距離0.11553990396805806 km
50325623313
存在メッシュ：50325623313値8.169491525423728距離0.0 km
50325623131
50325623131 is kddi none!
50325623113
存在メッシュ：50325623113値8.225832877567745距離0.46215938775195675 km
50325623111
存在メッシュ：50325623111値8.239918206270108距離0.5776991581275353 km
50325623131
50325623131 is kddi none!
50325613333
50325613333 is kddi none!
50325623133
50325623133 is kddi none!
50325613333
50325613333 is kddi none!
50325613331
存在メッシュ：50325613331値8.268088863291934距離0.808778695737822 km
50325613313
存在メッシュ：50325613313値8.282174184850833距離0.9243184075166689 km
50325613311
存在メッシュ：50325613311値8.296259504028558距離1.0398580997631206 km
50325613133
存在メッシュ：50325613133値8.310344827586206距離1.1553978279373909 km
50325623111
存在メッシュ：50325623111値8.239918206270108距離0.5776991581275353 km
33.7619791665:33.7723958335:132.788281250:132.788281250
50325623133
50325623133 is

50325623133
50325623133 is kddi none!
50325623311
50325623311 is kddi none!
50325623313
50325623313 is kddi none!
50325623131
存在メッシュ：50325623131値3.6720000738803646距離0.34661954238397397 km
50325623113
50325623113 is kddi none!
50325623111
50325623111 is kddi none!
50325623131
存在メッシュ：50325623131値3.6720000738803646距離0.34661954238397397 km
50325613333
50325613333 is kddi none!
50325623133
50325623133 is kddi none!
50325613333
50325613333 is kddi none!
50325613331
50325613331 is kddi none!
50325613313
存在メッシュ：50325613313値3.8920000552825798距離0.9243184075166689 km
50325613311
50325613311 is kddi none!
50325613133
50325613133 is kddi none!
50325623111
50325623111 is kddi none!
33.7619791665:33.7723958335:132.788281250:132.788281250
50325623133
50325623133 is kddi none!
50325623311
50325623311 is kddi none!
50325623313
50325623313 is kddi none!
50325623131
存在メッシュ：50325623131値3.7538182068520243距離0.34661954238397397 km
50325623113
50325623113 is kddi none!
50325623111
50325623111 is kddi none!
503

33.7619791665:33.7723958335:132.788281250:132.788281250
50325623133
50325623133 is kddi none!
50325623311
存在メッシュ：50325623311値5.325365358056664距離0.11553990396805806 km
50325623313
50325623313 is kddi none!
50325623131
存在メッシュ：50325623131値5.467621393539669距離0.34661954238397397 km
50325623113
存在メッシュ：50325623113値5.538749427385668距離0.46215938775195675 km
50325623111
存在メッシュ：50325623111値5.609877415065236距離0.5776991581275353 km
50325623131
存在メッシュ：50325623131値5.467621393539669距離0.34661954238397397 km
50325613333
存在メッシュ：50325613333値5.681005390719428距離0.6932389089691811 km
50325623133
50325623133 is kddi none!
50325613333
存在メッシュ：50325613333値5.681005390719428距離0.6932389089691811 km
50325613331
存在メッシュ：50325613331値5.752133388490808距離0.808778695737822 km
50325613313
存在メッシュ：50325613313値5.823261340097364距離0.9243184075166689 km
50325613311
存在メッシュ：50325613311値5.89438927967949距離1.0398580997631206 km
50325613133
存在メッシュ：50325613133値5.9655172413793105距離1.1553978279373909 km
50325623111
存在メッシュ：50325623111値5.60

50325623133
50325623133 is kddi none!
50325623311
50325623311 is kddi none!
50325623313
50325623313 is kddi none!
50325623131
50325623131 is kddi none!
50325623113
50325623113 is kddi none!
50325623111
50325623111 is kddi none!
50325623131
50325623131 is kddi none!
50325613333
50325613333 is kddi none!
50325623133
50325623133 is kddi none!
50325613333
50325613333 is kddi none!
50325613331
50325613331 is kddi none!
50325613313
50325613313 is kddi none!
50325613311
50325613311 is kddi none!
50325613133
50325613133 is kddi none!
50325623111
50325623111 is kddi none!
33.7619791665:33.7723958335:132.788281250:132.788281250
50325623133
50325623133 is kddi none!
50325623311
50325623311 is kddi none!
50325623313
50325623313 is kddi none!
50325623131
50325623131 is kddi none!
50325623113
50325623113 is kddi none!
50325623111
50325623111 is kddi none!
50325623131
50325623131 is kddi none!
50325613333
50325613333 is kddi none!
50325623133
50325623133 is kddi none!
50325613333
50325613333 is kddi 

50325623133
50325623133 is kddi none!
50325623311
50325623311 is kddi none!
50325623313
50325623313 is kddi none!
50325623131
50325623131 is kddi none!
50325623113
50325623113 is kddi none!
50325623111
50325623111 is kddi none!
50325623131
50325623131 is kddi none!
50325613333
50325613333 is kddi none!
50325623133
50325623133 is kddi none!
50325613333
50325613333 is kddi none!
50325613331
50325613331 is kddi none!
50325613313
50325613313 is kddi none!
50325613311
50325613311 is kddi none!
50325613133
50325613133 is kddi none!
50325623111
50325623111 is kddi none!
33.7619791665:33.7723958335:132.788281250:132.788281250
50325623133
50325623133 is kddi none!
50325623311
50325623311 is kddi none!
50325623313
50325623313 is kddi none!
50325623131
50325623131 is kddi none!
50325623113
50325623113 is kddi none!
50325623111
50325623111 is kddi none!
50325623131
50325623131 is kddi none!
50325613333
50325613333 is kddi none!
50325623133
50325623133 is kddi none!
50325613333
50325613333 is kddi 

In [59]:
connection.close()

In [54]:
#CreateGeojsonFile